### 1.二叉树按层遍历并收集节点

In [15]:

'''
同层节点在一个集合中，如：[a],[b,c],[d,e,f,g]...
bottom-up: [d,e,f,g], [b,c], [a],
组成：
       a
     ↙  ↘
     b    c
   ↙ ↘ ↙ ↘
   d  e f   g
简单思路：
1. 拿出此时队列的size
2. 每层逐次弹出size次，每次先添加左节点，然后再右节点 到队列。此层弹出的节点，共成为一层的节点集合
3. 如果想得到从下往上的每层节点，
3.1 可以将收集到的逐层节点集合的list，从[0]、[N-1]开始交换，0+=1，N-1-=1，直到list中间停止
3.2 或者在存储节点集合时，使用linkedlist，即收集第二层后，让第二层的next指向第一层（不能用list是因为ArrayList向前添加代价高(java中)，
    python中直接用insert的代价为O(n)，也很高
'''
#https://leetcode.com/problems/binary-tree-level-order-traversal-ii/

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

# class LinkedNode:
#     def __init__(self, val=0, next=None):
#         self.val = val
#         self.next

# class LinkedList:
#     def __init__(self, l=list(), next=None):
#         self.list = l
#         self.next = next
    
#     def __len__(self):
#         return len(self.list)
import heapq
   
def levelOrderBottom(root):
    ans = list()
    if not root:
        return ans
    queue = list()
    queue.append(root)
    while queue:
        curAns = list()
        size = len(queue)
        for i in range(size):
            curNode = queue.pop(0)
            curAns.append(curNode.val)
            if curNode.left:
                queue.append(curNode.left)
            if curNode.right:
                queue.append(curNode.right)
        ans.insert(0, curAns)
    return ans


### 2.判断是否平衡搜索二叉树

#### 2.1 判断是否是平衡树

In [21]:
#平衡树： 每一颗树 是否 |左树 - 右树| <= 1
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        
def process(root):
    if not root:
        return True,0 #isBalanced, height
    left_balanced, left_height = process(root.left)
    right_balanced, right_height = process(root.right)   
    cur_height = max(left_height, right_height) + 1
    cur_balanced = left_balanced & right_balanced & (abs(left_height - right_height) <= 1)
    return cur_balanced, cur_height

def isBalanced(root):
    return process(root)[0]


### 2.2 判断是否是搜索树

In [22]:
#搜索树：每一棵树 的是否 左树的所有节点 比我小，右树所有节点比我大
'''
第一种思路：中序遍历严格递增

第二种思路：递归判断
         左树是搜索二叉树 and 左树max < x
         右树是搜索二叉树 and 右树min > x
'''
def process(root):
    if not root:
        return None #交由上游自己判空
    leftInfo = process(root.left)
    rightInfo = process(root.right)
    max_val = root.val
    min_val = root.val
    if leftInfo is not None:
        max_val = max(leftInfo[1], max_val)
        min_val = min(leftInfo[2], min_val)
    if rightInfo is not None:
        max_val = max(rightInfo[1], max_val)
        min_val = min(rightInfo[2], min_val)
    isSearch = True
    #一旦发现false 实际上可以跳过后续直接返回false
    if leftInfo is not None and (not leftInfo[0]):
        isSearch = False
    if rightInfo is not None and (not rightInfo[0]):
        isSearch = False
    leftMaxLessRoot = True if not leftInfo else leftInfo[1] < root.val
    rightMinGreaterRoot = True if not rightInfo else rightInfo[2] > root.val
    if (not leftMaxLessRoot) | (not rightMinGreaterRoot):
        isSearch = False
    return isSearch, max_val, min_val

def isValidBST(root):
    if not root:
        return True
    return process(root)[0]

### 3.能否组成路径和

In [24]:
#https://leetcode.com/problems/path-sum/
#某一条路径，头节点到某个叶节点的所有途径节点和,判断是否存在路径和

        

# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def hasPathSum(self, root, targetSum):
        if not root:
            return False
        
        def process(root,preSum,tarSum):
            if (not root.left) and (not root.right):
                if root.val + preSum == tarSum:
                    return True
            #root非叶节点
            preSum += root.val
            left = False
            right = False
            if root.left:
                left = process(root.left, preSum, tarSum)
            if root.right:
                right = process(root.right, preSum, tarSum)
            return left | right
        
        
        isSum = process(root, 0, targetSum)
        return isSum

### 4.返回所有存在路径和的路径

In [25]:
#https://leetcode.com/problems/path-sum-ii/
#某一条路径，头节点到某个叶节点的所有途径节点和， 返回所有可以的路径

# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def pathSum(self, root, targetSum):
        ans = list()
        if not root:
            return ans
        
        def process(root, path, preSum, targetSum, ans):
            if (not root.left) and (not root.right):
                if(preSum + root.val == targetSum):
                    path.append(root.val)
                    ans.append(path[:])
                    #恢复现场
                    path.pop()
                return
            # root 非叶节点
            path.append(root.val)
            preSum += root.val
            if root.left:
                process(root.left, path, preSum, targetSum, ans)
            if root.right:
                process(root.right, path, preSum, targetSum, ans) 
            #恢复现场
            path.pop()
            return 

        process(root, list(), 0, targetSum, ans)
        return ans
        